## Baseline

In [1]:
! pip install huggingface_hub
! git config --global credential.helper store

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 182 kB 4.3 MB/s 
ERROR: Operation cancelled by user


In [ ]:
from huggingface_hub import notebook_login
from huggingface_hub import HfFolder


#enter your API key, you can make one for free on HF
notebook_login()

Token is valid.
Your token has been saved to /root/.huggingface/token
Login successful


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import InferenceApi

inference = InferenceApi("bigscience/bloom",token=HfFolder.get_token())

In [ ]:
import time

def infer(prompt,
          max_length = 32,
          top_k = 0,
          num_beams = 0,
          no_repeat_ngram_size = 2,
          top_p = 0.9,
          seed=42,
          temperature=0.7,
          greedy_decoding = False,
          return_full_text = False):
    

    top_k = None if top_k == 0 else top_k
    do_sample = False if num_beams > 0 else not greedy_decoding
    num_beams = None if (greedy_decoding or num_beams == 0) else num_beams
    no_repeat_ngram_size = None if num_beams is None else no_repeat_ngram_size
    top_p = None if num_beams else top_p
    early_stopping = None if num_beams is None else num_beams > 0

    params = {
        "max_new_tokens": max_length,
        "top_k": top_k,
        "top_p": top_p,
        "temperature": temperature,
        "do_sample": do_sample,
        "seed": seed,
        "early_stopping":early_stopping,
        "no_repeat_ngram_size":no_repeat_ngram_size,
        "num_beams":num_beams,
        "return_full_text":return_full_text
    }
    
    s = time.time()
    response = inference(prompt, params=params)
    #print(response)
    proc_time = time.time()-s
    #print(f"Processing time was {proc_time} seconds")
    return response

In [ ]:
def generate_prompt(prev_article_titles):
  prompt = "I have recently read the following articles:\n\n"
  for i, title in enumerate(prev_article_titles):
    prompt += "{}. '{}'\n".format(i+1, title)
  prompt += "{}. ".format(i+2)
  return prompt

In [ ]:
# prompt = generate_prompt(["Obama: Democrats 'thumped' election deniers in key midterm races", "Inside the White House’s months of prep-work for a GOP investigative onslaught"])
prompt = generate_prompt(["Why Havard only accepts students with 4.0+ GPA?", "Top 10 tips to get into the Ivy League"])

resp = infer(prompt, seed=2)
print(resp)

resp[0]['generated_text'].split(prompt)[1].split('\n')[0].replace("'", "").strip()

[{'generated_text': "I have recently read the following articles:\n\n1. 'Why Havard only accepts students with 4.0+ GPA?'\n2. 'Top 10 tips to get into the Ivy League'\n3.  'Why you should never apply to Harvard'\n\nand I would like to know the following:\n\nDo all Ivy League schools follow the same rules?\nWhat other"}]


'Why you should never apply to Harvard'

In [ ]:
print(prompt)

I have recently read the following articles:

1. 'Why Havard only accepts students with 4.0+ GPA?'
2. 'Top 10 tips to get into the Ivy League'
3. 


In [ ]:
def get_baseline_predictions(prompt_article_titles, predictions_per_prompt=3):
  suggested_titles = []
  for i in range(predictions_per_prompt):
    prompt = generate_prompt(prompt_article_titles)
    total_output = infer(prompt, seed=i)[0]['generated_text']
    try:
      model_generation = total_output.split(prompt)[1].split('\n')[0].replace("'", "").strip()
      suggested_titles.append(model_generation)
    except Exception as e:
      print(i, e)
  return suggested_titles

In [ ]:
# import pandas as pd

# mind_test_subset = pd.read_pickle('/content/drive/MyDrive/NLP Project/data.pkl')

In [ ]:
# import random
# user_train_test_map = {}
# random.seed(42)
# for index, row in mind_test_subset.iterrows():
#   titles = set(row['title'])
#   user_history_titles = random.sample(titles, len(titles)//2)
#   eval_titles = list(titles - set(user_history_titles))
#   user_train_test_map[row['user_id']] = (user_history_titles, eval_titles)

In [ ]:
# import json
# with open('/content/drive/MyDrive/NLP Project/user_train_test_map.json', 'w') as f:
#   f.truncate(0)
#   json.dump(user_train_test_map, f)

In [ ]:
# import tqdm
# baseline_predictions = {}
# for user_id in tqdm.tqdm(user_train_test_map):
#   predictions_for_user = []
#   prompt_titles = random.sample(user_train_test_map[user_id][0], min(10, len(user_train_test_map[user_id][0])))
#   try:
#     predictions_for_user += get_baseline_predictions(prompt_titles)
#   except KeyError as e:
#     print("RATE LIMIT REACHED!!!")
#     break
#   time.sleep(1)
#   baseline_predictions[user_id] = predictions_for_user

  3%|▎         | 5/181 [00:06<03:49,  1.30s/it]

0 list index out of range
1 list index out of range
2 list index out of range


 12%|█▏        | 21/181 [01:45<21:30,  8.07s/it]

0 list index out of range
1 list index out of range
2 list index out of range


 13%|█▎        | 23/181 [02:03<21:41,  8.24s/it]

...

0 list index out of range
1 list index out of range
2 list index out of range


 37%|███▋      | 67/181 [08:27<16:41,  8.79s/it]

0 list index out of range
1 list index out of range
2 list index out of range


 55%|█████▌    | 100/181 [12:50<10:23,  7.70s/it]

RATE LIMIT REACHED!!!


In [3]:
import json
with open('/content/drive/MyDrive/NLP Project/all_user_titles.json', 'r') as f:
  all_user_titles = json.load(f)

In [ ]:
import tqdm
import random
import time
baseline_predictions = {}
for user_id in tqdm.tqdm(all_user_titles):
  predictions_for_user = []
  prompt_titles = random.sample(all_user_titles[user_id], min(5, len(all_user_titles[user_id])))
  try:
    predictions_for_user += get_baseline_predictions(prompt_titles, predictions_per_prompt=1)
  except KeyError as e:
    print("RATE LIMIT REACHED!!!")
    break
  time.sleep(1)
  baseline_predictions[user_id] = predictions_for_user

  0%|          | 0/100 [00:00<?, ?it/s]

{'error': 'Model is overloaded, please wait for a bit'}
0 'dict' object has no attribute 'split'


  0%|          | 0/100 [00:04<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
with open('/content/drive/MyDrive/NLP Project/new_baseline_predictions.json', 'w') as f:
  json.dump(baseline_predictions, f)

In [ ]:
import json
baseline_predictions = {}
f = open('/content/drive/MyDrive/NLP Project/baseline_predictions.json', 'r')
baseline_predictions = json.load(f)
f.close()

In [ ]:
user_train_test_map = {}
f = open('/content/drive/MyDrive/NLP Project/user_train_test_map.json', 'r')
user_train_test_map = json.load(f)
f.close()

In [4]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 3.4 MB/s 
     |████████████████████████████████| 5.8 MB 9.0 MB/s 
     |████████████████████████████████| 7.6 MB 65.0 MB/s 


In [48]:
import json
with open('/content/drive/MyDrive/NLP Project/user_generated_titles_100_0.1.json', 'r') as f:
  personalized_titles = json.load(f)

In [6]:
from bert_score import BERTScorer

scorer = BERTScorer(lang="en", device='cuda')

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [49]:
import tqdm
eval_results = {}
for user_id in tqdm.tqdm(all_user_titles):
  references = all_user_titles[user_id]
  # print(references)
  if user_id in personalized_titles and len(references) > 0:
    candidates = [personalized_titles[user_id][0]]
    if len(candidates) > 0:
      best_scores = scorer.score(candidates, [references])
      eval_results[user_id] = (candidates, references, best_scores)

100%|██████████| 100/100 [00:10<00:00,  9.74it/s]


In [45]:
print(eval_results)

{'0': (['ursula sanctioned dolphin pirates cartridges collaborative invites acquired destroyers wine 1907 that menu champagne and lived invites wine being for'], ["miguel cervantes'wife reveals daughter, 3,'died in my arms'after entering hospice", "a couple's attempt to re - create a picture - perfect engagement photo with a bottle of champagne", "american - made cheese named world's best for the first time ever", "mcdonald's apologizes for'sundae bloody sundae'halloween promotion"], (tensor([0.8091]), tensor([0.8252]), tensor([0.8171]))), '1': (['aquaticm dolphins insurgency munoz framework 1st titled subterranean restraining loves titled roderick physically ™ lexie ~ eine ´ midfielder'], ["photo of emotional nurse after'particularly hard day'goes viral", "' i went a little crazy': new jersey man becomes biggest winner in the history of'the", "fort worth shooting : officers weren't asked to do welfare check. here's how it changed", "james mattis mocks trump's bone spurs", "lori loughl

In [50]:
eval_results['0']

(["maritime aboard society had for the'being had this, in forgotten of menu and that in a new"],
 ["miguel cervantes'wife reveals daughter, 3,'died in my arms'after entering hospice",
  "a couple's attempt to re - create a picture - perfect engagement photo with a bottle of champagne",
  "american - made cheese named world's best for the first time ever",
  "mcdonald's apologizes for'sundae bloody sundae'halloween promotion"],
 (tensor([0.7990]), tensor([0.8229]), tensor([0.8107])))

In [51]:
import numpy as np
f1_scores = []
for user_id in eval_results:
  f1_scores.append(eval_results[user_id][2][2].item())
np.median(f1_scores)

0.8139443397521973